# 📌 Laboratorio: Redes | Múltiples VPC Networks

## Descripción General
Las **Redes de Nube Privada Virtual (VPC)** permiten mantener entornos aislados dentro de una infraestructura de nube más grande, ofreciendo control detallado sobre la protección de datos, el acceso a la red y la seguridad de las aplicaciones. En este laboratorio, crearás dos redes VPC en modo personalizado (**managementnet** y **privatenet**) con reglas de firewall e instancias de máquinas virtuales (VM), y probarás la conectividad entre ellas.

💡 **Analogía contable**: Una **VPC** es como una sucursal contable dentro de una empresa. Cada sucursal tiene sus propias reglas (firewalls) y registros internos (subredes), asegurando que los datos contables no se mezclen con otras sucursales.

### Objetivos del Laboratorio
- Crear redes **VPC** en modo personalizado con reglas de firewall.
- Crear instancias de máquinas virtuales (**VM**) usando **Compute Engine**.
- Explorar la conectividad entre instancias de VM en diferentes redes **VPC**.
- Crear una instancia de VM con múltiples interfaces de red.

💡 **Contexto empresarial**: Configurar redes **VPC** es como establecer departamentos contables independientes con políticas de acceso específicas, garantizando seguridad y cumplimiento normativo.

Para más información, consulta la [Documentación de Google Cloud VPC](https://cloud.google.com/vpc/docs).

## 🚀 Configuración y Requisitos

### Activación de Cloud Shell
**Cloud Shell** es una máquina virtual preconfigurada para ejecutar comandos y gestionar recursos en **Google Cloud**.

**Pasos**:
1. Haz clic en el ícono de **Cloud Shell** en la barra superior.
2. Autoriza su uso.
3. Verifica el ID del proyecto activo en la sesión.

**Comandos iniciales**:
```bash
gcloud auth list
```
**Explicación**: Lista las cuentas activas autenticadas.

```bash
gcloud config list project
```
**Explicación**: Muestra el ID del proyecto actual.

💡 **Contexto empresarial**: Como verificar las credenciales y el proyecto contable activo antes de registrar transacciones en un ERP.

## 🔒 Tarea 1: Crear Redes VPC en Modo Personalizado con Reglas de Firewall

### Crear la Red managementnet (desde la consola)
1. Navega a **VPC network > VPC networks**.
2. Haz clic en **Create VPC Network**.
3. Configura:
   - **Nombre**: `managementnet`
   - **Subnet mode**: Custom
   - **Subred**:
     - Nombre: `managementsubnet-1`
     - Región: `<Region_1>` (ej. `us-central1`)
     - Rango IPv4: `10.130.0.0/20`
4. Haz clic en **Done** y luego en **Create**.

**Comando equivalente**:
```bash
gcloud compute networks create managementnet --project=qwiklabs-gcp-03-08f5a30a4dc2 --subnet-mode=custom --mtu=1460 --bgp-routing-mode=regional --bgp-best-path-selection-mode=legacy
gcloud compute networks subnets create managementsubnet-1 --project=qwiklabs-gcp-03-08f5a30a4dc2 --range=10.130.0.0/20 --stack-type=IPV4_ONLY --network=managementnet --region=us-central1
```
**Explicación**:
- Crea una red **VPC** personalizada (`managementnet`) y una subred (`managementsubnet-1`) con un rango de 4,096 IPs.
- **Nota**: Reemplaza `qwiklabs-gcp-03-08f5a30a4dc2` con tu ID de proyecto y `<Region_1>` con la región asignada (ej. `us-central1`).

💡 **Analogía contable**: Crear una red es como abrir una sede contable con su propio plan de cuentas (subredes) y controles internos (firewall).

### Crear la Red privatenet (desde Cloud Shell)
```bash
gcloud compute networks create privatenet --subnet-mode=custom
gcloud compute networks subnets create privatesubnet-1 --network=privatenet --region=<Region_1> --range=172.16.0.0/24
gcloud compute networks subnets create privatesubnet-2 --network=privatenet --region=<Region_2> --range=172.20.0.0/20
```
**Explicación**:
- Crea una red **VPC** personalizada (`privatenet`).
- Crea dos subredes: `privatesubnet-1` (256 IPs) en `<Region_1>` (ej. `us-central1`) y `privatesubnet-2` (4,096 IPs) en `<Region_2>` (ej. `us-east1`).
- **Nota**: Reemplaza `<Region_1>` y `<Region_2>` con las regiones asignadas.

💡 **Analogía contable**: Las subredes son como divisiones internas (tesorería, cuentas por pagar) con rangos de trabajo definidos.

### Verificar Redes y Subredes
```bash
gcloud compute networks list
gcloud compute networks subnets list --sort-by=NETWORK
```
**Explicación**:
- Lista todas las redes **VPC** y subredes, ordenadas por red.
- **Nota**: Las redes `default` y `mynetwork` son de modo automático; `managementnet` y `privatenet` son personalizadas.

En la consola, navega a **VPC network > VPC networks** para verificar.

💡 **Contexto empresarial**: Como revisar el organigrama contable para confirmar las sedes y departamentos registrados.

### Crear Reglas de Firewall para managementnet
1. Navega a **VPC network > Firewall > Create Firewall Rule**.
2. Configura:
   - **Nombre**: `managementnet-allow-icmp-ssh-rdp`
   - **Red**: `managementnet`
   - **Targets**: All instances in the network
   - **Source filter**: IPv4 Ranges
   - **Rango IPv4**: `0.0.0.0/0`
   - **Protocolos y puertos**:
     - TCP: `22` (SSH), `3389` (RDP)
     - ICMP
3. Haz clic en **Create**.

**Comando equivalente**:
```bash
gcloud compute firewall-rules create managementnet-allow-icmp-ssh-rdp --project=qwiklabs-gcp-03-08f5a30a4dc2 --direction=INGRESS --priority=1000 --network=managementnet --action=ALLOW --rules=tcp:22,tcp:3389,icmp --source-ranges=0.0.0.0/0
```
**Explicación**:
- Crea una regla de firewall para permitir tráfico ICMP, SSH y RDP desde cualquier IP.
- **Nota**: Reemplaza `qwiklabs-gcp-03-08f5a30a4dc2` con tu ID de proyecto.

💡 **Analogía contable**: Como definir quién puede acceder a la bóveda contable (solo auditores autorizados).

### Crear Reglas de Firewall para privatenet
```bash
gcloud compute firewall-rules create privatenet-allow-icmp-ssh-rdp --direction=INGRESS --priority=1000 --network=privatenet --action=ALLOW --rules=icmp,tcp:22,tcp:3389 --source-ranges=0.0.0.0/0
```
**Explicación**:
- Permite tráfico ICMP, SSH (puerto 22) y RDP (puerto 3389) en `privatenet` desde cualquier IP.

💡 **Contexto empresarial**: Como establecer políticas para que solo ciertos roles accedan a datos financieros.

### Verificar Reglas de Firewall
```bash
gcloud compute firewall-rules list --sort-by=NETWORK
```
**Explicación**: Lista todas las reglas de firewall, agrupadas por red.

En la consola, navega a **VPC network > Firewall** para verificar.

💡 **Contexto empresarial**: Como auditar las políticas de acceso en un sistema ERP.

## 🔧 Tarea 2: Creación y Gestión de Instancias de Máquina Virtual (VM)

### Navegación para Crear Instancias VM
1. Navega a **Compute Engine > VM instances**.
2. Observa las instancias preexistentes: `mynet-vm-1` y `mynet-vm-2`.
3. Haz clic en **Create Instance**.

💡 **Contexto empresarial**: Como registrar un nuevo proyecto contable en el sistema.

### Configuración de la Instancia managementnet-vm-1
| Propiedad | Valor |
|-----------|-------|
| **Nombre** | `managementnet-vm-1` |
| **Región** | `<US_Region>` (ej. `us-central1`) |
| **Zona** | `<US_Zone>` (ej. `us-central1-a`) |
| **Serie** | E2 |
| **Tipo de máquina** | `e2-micro` |
| **Red** | `managementnet` |
| **Subred** | `managementsubnet-1` |

Haz clic en **Create**.

💡 **Analogía contable**: Como asignar un nuevo centro de costo a una sede contable específica.

### Creación de la Instancia privatenet-vm-1
```bash
gcloud compute instances create privatenet-vm-1 --zone=<US_Zone> --machine-type=e2-micro --subnet=privatesubnet-1
```
**Explicación**:
- Crea una VM en `privatesubnet-1` con tipo `e2-micro`.
- **Nota**: Reemplaza `<US_Zone>` con la zona asignada (ej. `us-central1-a`).

💡 **Contexto empresarial**: Como registrar un proyecto en un departamento específico con recursos limitados.

### Verificación de Instancias
```bash
gcloud compute instances list --sort-by=ZONE
```
**Explicación**: Lista todas las instancias, ordenadas por zona.

En la consola, navega a **Compute Engine > VM instances**, activa la columna **Network** en **Column display options**.

💡 **Contexto empresarial**: Como revisar el registro de proyectos contables en un ERP.

| Paso | Descripción | Analogía Contable |
|------|-------------|------------------|
| **Navegar a Instancias VM** | Ir a Compute Engine > VM instances | Acceder al módulo de proyectos en contabilidad |
| **Crear instancia VM** | Configurar nombre, región, zona, red/subred | Registrar un proyecto en una sucursal |
| **Crear por línea de comando** | Usar `gcloud compute instances create` | Ejecutar un procedimiento contable vía script |
| **Verificar instancia** | Listar instancias para confirmar | Verificar registros en el ERP |
| **Distribución en redes y zonas** | Instancias en distintas redes y zonas | Proyectos en distintos departamentos y oficinas |

## 🌐 Tarea 3: Explorar la Conectividad entre Instancias de VM

### Verificación de Conectividad a través de Direcciones IP Externas
1. Navega a **Compute Engine > VM instances**.
2. Anota las IPs externas de: `mynet-vm-2`, `managementnet-vm-1`, `privatenet-vm-1`.
3. Desde la terminal SSH de `mynet-vm-1`, ejecuta:
```bash
ping -c 3 <mynet-vm-2 external IP>
ping -c 3 <managementnet-vm-1 external IP>
ping -c 3 <privatenet-vm-1 external IP>
```
**Explicación**:
- Prueba la conectividad a IPs externas, que debería funcionar gracias a las reglas de firewall (`ICMP` permitido).
- **Nota**: Reemplaza `<... external IP>` con las IPs correspondientes.

💡 **Contexto empresarial**: Como enviar correos a todas las sedes, permitido por políticas corporativas.

### Verificación de Conectividad a través de Direcciones IP Internas
1. Anota las IPs internas de: `mynet-vm-2`, `managementnet-vm-1`, `privatenet-vm-1`.
2. Desde la terminal SSH de `mynet-vm-1`, ejecuta:
```bash
ping -c 3 <mynet-vm-2 internal IP>
```
**Resultado**: Funciona, ya que `mynet-vm-1` y `mynet-vm-2` están en la misma red **VPC**.

```bash
ping -c 3 <managementnet-vm-1 internal IP>
ping -c 3 <privatenet-vm-1 internal IP>
```
**Resultado**: Falla (100% de pérdida de paquetes), ya que están en redes **VPC** diferentes.

💡 **Analogía contable**: Como intentar compartir datos contables entre departamentos sin un acuerdo formal (VPC Peering o VPN).

**Consideración técnica**: Las redes **VPC** son aisladas por defecto. Para conectarlas, se necesita **VPC Peering** o una **VPN**.

| Elemento Evaluado | Detalle | Analogía Contable |
|-------------------|---------|------------------|
| **Comunicación externa (IP pública)** | Accesible si el firewall lo permite | Enviar correos a cualquier sede |
| **Comunicación interna (IP privada)** | Solo dentro de la misma VPC | Compartir datos dentro de un departamento |
| **Restricción entre redes VPC** | VPCs aisladas por defecto | Departamentos sin acceso a otros balances |
| **Excepciones** | VPC Peering o VPN | Auditoría compartida o sistema integrado |
| **Factores como zonas o regiones** | No afectan dentro de una VPC | Sucursales bajo el mismo sistema contable |

## 🛠️ Tarea 4: Crear una Instancia de VM con Múltiples Interfaces de Red

### Introducción
Cada instancia en una **VPC** tiene una interfaz de red predeterminada, pero se pueden añadir hasta 8 interfaces (según el tipo de máquina). Esto permite conectar una VM a múltiples redes **VPC**.

💡 **Analogía contable**: Como un contador que trabaja con varios departamentos (contabilidad, administración, operaciones), cada uno con su propio sistema.

### Crear la Instancia vm-appliance

1. Navega a **Compute Engine > VM instances > Create Instance**.

2. Configura:

| Propiedad | Valor |
|-----------|-------|
| **Nombre** | `vm-appliance` |
| **Región** | `<US_Region>` (ej. `us-central1`) |
| **Zona** | `<US_Zone>` (ej. `us-central1-a`) |
| **Serie** | E2 |
| **Tipo de máquina** | `e2-standard-4` |

3. En **Networking > Network interfaces**, configura:
   - **Interfaz 1 (nic0)**:
     - Red: `privatenet`
     - Subred: `privatesubnet-1`
   - **Interfaz 2 (nic1)**:
     - Red: `managementnet`
     - Subred: `managementsubnet-1`
   - **Interfaz 3 (nic2)**:
     - Red: `mynetwork`
     - Subred: `mynetwork`
4. Haz clic en **Create**.

**Nota**: Los rangos CIDR de las subredes no deben superponerse.

💡 **Contexto empresarial**: Como un contador que gestiona múltiples sistemas contables sin mezclar datos.

### Verificar Interfaces de Red desde la Consola
1. Navega a **Compute Engine > VM instances**.
2. Haz clic en `vm-appliance` y verifica:
   - **nic0**: Conectada a `privatesubnet-1` (rango: `172.16.0.0/24`).
   - **nic1**: Conectada a `managementsubnet-1` (rango: `10.130.0.0/20`).
   - **nic2**: Conectada a `mynetwork` (rango: `10.128.0.0/20`).

💡 **Contexto empresarial**: Como confirmar que un contador tiene acceso a los sistemas correctos.

### Verificar Interfaces desde la Terminal
1. En `vm-appliance`, haz clic en **SSH**.
2. Ejecuta:
```bash
sudo ifconfig
```
**Explicación**: Lista las interfaces de red activas (`eth0`, `eth1`, `eth2`).

💡 **Analogía contable**: Como revisar las cuentas bancarias asignadas a un contador.

### Explorar la Conectividad de las Interfaces
1. Anota las IPs internas de: `privatenet-vm-1`, `managementnet-vm-1`, `mynet-vm-1`, `mynet-vm-2`.
2. Desde la terminal SSH de `vm-appliance`, ejecuta:
```bash
ping -c 3 <privatenet-vm-1 internal IP>
ping -c 3 privatenet-vm-1
ping -c 3 <managementnet-vm-1 internal IP>
ping -c 3 <mynet-vm-1 internal IP>
ping -c 3 <mynet-vm-2 internal IP>
```
**Resultados**:
- Funciona para `privatenet-vm-1`, `managementnet-vm-1`, `mynet-vm-1` (mismas redes que las interfaces).
- Falla para `mynet-vm-2` (subred no listada en las rutas).

3. Verifica las rutas:
```bash
ip route
```
**Resultado esperado**:
```
default via 172.16.0.1 dev eth0
10.128.0.0/20 via 10.128.0.1 dev eth2
10.128.0.1 dev eth2 scope link
10.130.0.0/20 via 10.130.0.1 dev eth1
10.130.0.1 dev eth1 scope link
172.16.0.0/24 via 172.16.0.1 dev eth0
172.16.0.1 dev eth0 scope link
```
**Explicación**:
- Solo `eth0` tiene una ruta por defecto.
- El tráfico a `mynet-vm-2` (subred `10.132.0.0/20`) falla porque no está en las rutas.
- **Solución**: Configurar **policy routing** (ver [Documentación de Google Cloud Policy Routing](https://cloud.google.com/vpc/docs/configure-policy-based-routing)).

💡 **Analogía contable**: Como un contador con acceso a varias cuentas, pero solo una permite transferencias externas. Si una cuenta no está autorizada, las operaciones fallan.

## 🚀 Conclusión

Este laboratorio te permitió:
1. Crear redes **VPC** personalizadas (`managementnet`, `privatenet`) con subredes y reglas de firewall.
2. Configurar instancias de **VM** en **Compute Engine**.
3. Probar la conectividad entre VMs en diferentes redes **VPC**.
4. Crear una VM con múltiples interfaces de red para conectar varias **VPC**.

💡 **Beneficio empresarial**: Configurar redes **VPC** y VMs permite gestionar datos financieros de forma segura, aislando departamentos contables y optimizando el acceso, similar a un ERP bien estructurado.

Para más información, consulta la [Documentación de Google Cloud VPC](https://cloud.google.com/vpc/docs).

**Nota sobre la imagen**: [Insertar imagen `image1.png` aquí, que ilustra el esquema de las redes VPC y sus conexiones.]